In [1]:
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing
from tensorflow.keras import preprocessing , utils
import os
import yaml
import json
import pandas as pd
from tensorflow.keras.callbacks import ModelCheckpoint
import pandas as pd

In [2]:
with open("Question & Answer.json",encoding='utf-8', errors='ignore') as json_data:
     chats = json.load(json_data, strict=False)

In [3]:
len(chats)

137052

In [4]:
chats[:1]

[{'question': 'zirconium dental implants. how common is it used now. is there any advantages or benefits over titanium implants. cons & pros please. thanks.',
  'short_answer': '\nDental implants\n',
  'answer': 'a majority of the dental implants placed are titanium. they are highly successful with many years use ; many studies much lower in cost ; have many restorative options. zirconia implants are newer fewer studies on success lesser restorative options. however they can be more aesthetic in certain anterior(front) situations. let your dentist/oral surgeon chose what they feel will be best for you.',
  'tags': ['dentistry'],
  'url': 'https://www.healthtap.com/user_questions/1160508-zirconium-dental-implants-how-common-is-it-used-now-is-there-any-advantages-or-benefits-over-titan'}]

In [5]:
questions_for_token = list()
answers_for_token = list()

for con in chats:
    questions_for_token.append(con['question'])
    answers_for_token.append(con['answer'])

In [6]:
final_QA = list()
questions = list()
answers = list()

In [7]:
count=0
for con in chats:
    questions.append(con['question'])
    answers.append(con['answer'])
    count+=1
    if(count>=100):
        final_QA.append([questions,answers])
        count=0
        questions = list()
        answers = list()

In [8]:
del(chats)

In [9]:
len(final_QA)

1370

In [10]:
len(final_QA[0][0])

100

In [11]:
len(final_QA[0][1])

100

In [9]:
embed_size = 100 # how big is each word vector
max_features=60000
maxlen=100

In [10]:
import re
def processTweet(chat):
    chat = chat.lower()
    chat = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',chat)
    chat = re.sub('@[^\s]+','',chat)
    chat = re.sub('[\s]+', ' ', chat)
    chat = re.sub(r'#([^\s]+)', r'\1', chat)
    chat = re.sub(r'[\.!:\?\-\'\"\\/]', r'', chat)
    chat = chat.strip('\'"')
    return chat

In [11]:
def getStopWordList(stopWordListFileName):
    #read the stopwords file and build a list
    stopWords = []
    fp = open(stopWordListFileName, 'r')
    line = fp.readline()
    while line:
        word = line.strip()
        stopWords.append(word)
        line = fp.readline()
    fp.close()
    return stopWords
#stopWords = getStopWordList('StopWords.txt')

In [12]:
def replaceTwoOrMore(s):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{2,}", re.DOTALL)
    return pattern.sub(r"\1", s)
def getFeatureVector(chat):
    chat=processTweet(chat)
    featureVector = []
    #split tweet into words
    words = chat.split()
    for w in words:
        #replace two or more with two occurrences
        w = replaceTwoOrMore(w)
        #strip punctuation
        w = w.strip('\'"?,.')
        #check if the word stats with an alphabet
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
        #ignore if it is a stop word
        if(val is None):
            continue
        else:
            featureVector.append(w.lower())
    return " ".join(list(featureVector))

In [13]:
def stringCropper(string):
    if("." in string):
        split_string=string.strip().split(".")
        if(len(split_string)>1 ):
            split_string=".".join(split_string[:2])
            return split_string
        else:
            return ".".join(split_string)
    else:
        return string

def stringCropperQues(string):
    if("." in string):
        split_string=string.strip().split(".")
        if(len(split_string)>0 ):
            split_string=".".join(split_string[:1])
            return split_string
        else:
            return ".".join(split_string)
    else:
        return string

In [14]:
def emb_mat(nb_words):
    EMBEDDING_FILE="glove.6B.100d.txt"
    def get_coefs(word,*arr): 
        return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE, encoding="utf8"))

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    emb_mean,emb_std

    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words+1, embed_size))
    for word, i in word_index.items():
        if (i >= max_features) or i==nb_words:
            continue
        embedding_vector = embeddings_index.get(word) #here we will get embedding for each word from GloVe
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            
    return embedding_matrix

In [15]:
def tokenized_data(questions,answers,VOCAB_SIZE,tokenizer):
    # encoder_input_data
    import numpy as np
    tokenized_questions = tokenizer.texts_to_sequences( questions )
    maxlen_questions = max( [ len(x) for x in tokenized_questions ] )
    padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions , maxlen=maxlen , padding='post' )
    encoder_input_data = np.array( padded_questions )
    #print( encoder_input_data.shape , maxlen_questions )

    # decoder_input_data
    tokenized_answers = tokenizer.texts_to_sequences( answers )
    maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
    padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen , padding='post' )
    decoder_input_data = np.array( padded_answers )
    #print( decoder_input_data.shape , maxlen_answers )

    # decoder_output_data
    tokenized_answers = tokenizer.texts_to_sequences( answers )
    for i in range(len(tokenized_answers)) :
        tokenized_answers[i] = tokenized_answers[i][1:] # remove <start> take rest
    padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen , padding='post' )
    onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE)
    decoder_output_data = np.array( onehot_answers )
    #print( decoder_output_data.shape )
    
    return [encoder_input_data,decoder_input_data,decoder_output_data,maxlen_questions,maxlen_answers]

In [16]:
answers_for_token=pd.DataFrame(answers_for_token, columns=["Ans"])
questions_for_token=pd.DataFrame(questions_for_token, columns=["Question"])
answers_for_token["cleanedAns"]=answers_for_token["Ans"].apply(stringCropper)
questions_for_token["cleanedQues"]=questions_for_token["Question"].apply(stringCropperQues)
answers_for_token["TokAns"]=answers_for_token["cleanedAns"].apply(getFeatureVector)
questions_for_token["TokQues"]=questions_for_token["cleanedQues"].apply(getFeatureVector)

answers_for_token=np.array(answers_for_token["TokAns"])
questions_for_token=np.array(questions_for_token["TokQues"])

answers_with_tags = list()
for i in range( len( answers_for_token ) ):
    if type( answers_for_token[i] ) == str:
        answers_with_tags.append( answers_for_token[i] )
    else:
        print(questions_for_token[i])
        print(answers_for_token[i])
        print(type(answers_for_token[i]))
        questions_for_token.pop(i)

answers_for_token = list()
for i in range( len( answers_with_tags ) ) :
    answers_for_token.append( '<START> ' + answers_with_tags[i] + ' <END>' )

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions_for_token+answers_for_token)

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))

embedding_matrix=emb_mat(nb_words)

VOCAB_SIZE = len( tokenizer.word_index )+1

C:\Users\pranj\anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3331: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [17]:
VOCAB_SIZE, nb_words

(58069, 58068)

In [18]:
def enable_gpu():
    config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
    sess = tf.compat.v1.Session(config=config) 
    #keras.backend.set_session(sess)
    tf.compat.v1.keras.backend.set_session(
        sess
    )
    gpu_options = tf.compat.v1.GPUOptions(allow_growth=True, per_process_gpu_memory_fraction=0.333)
    session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

In [19]:
def get_model(nb_words,embed_size,embedding_matrix):
    #Defining model architecture for seq2seq Model
    # return_seq=T -> gives hidden state o/p at each time step
    #return_state =T -> Gives hidden state and cell o/p of only last time step
    #if both options not given -> hidden state of last time step
    encoder_inputs = tf.keras.layers.Input(shape=( None , ))
    encoder_embedding = tf.keras.layers.Embedding( nb_words+1, embed_size , mask_zero=True ,weights=[embedding_matrix]) (encoder_inputs)
    encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
    encoder_states = [ state_h , state_c ]

    decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
    decoder_embedding = tf.keras.layers.Embedding( nb_words+1, embed_size , mask_zero=True,weights=[embedding_matrix]) (decoder_inputs)
    decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
    decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )

    decoder_dense = tf.keras.layers.Dense( nb_words+1 , activation=tf.keras.activations.softmax ) 
    output = decoder_dense ( decoder_outputs )

    model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
    return model,encoder_inputs,encoder_states,decoder_lstm,decoder_embedding,decoder_dense,decoder_inputs

In [20]:
# define the checkpoint
filepath = "model_DoctorChat.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [21]:
def prepare_data(questions,answers):
    answers=pd.DataFrame(answers, columns=["Ans"])
    questions=pd.DataFrame(questions, columns=["Question"])

    answers["cleanedAns"]=answers["Ans"].apply(stringCropper)
    questions["cleanedQues"]=questions["Question"].apply(stringCropperQues)
    answers["TokAns"]=answers["cleanedAns"].apply(getFeatureVector)
    questions["TokQues"]=questions["cleanedQues"].apply(getFeatureVector)

    answers=np.array(answers["TokAns"])
    questions=np.array(questions["TokQues"])

    answers_with_tags = list()
    for i in range( len( answers ) ):
        if type( answers[i] ) == str:
            answers_with_tags.append( answers[i] )
        else:
            print(questions[i])
            print(answers[i])
            print(type(answers[i]))
            questions.pop(i)

    answers = list()
    for i in range( len( answers_with_tags ) ) :
        answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )

    tok_out=tokenized_data(questions,answers,VOCAB_SIZE,tokenizer)
    encoder_input_data=tok_out[0]
    decoder_input_data=tok_out[1]
    decoder_output_data=tok_out[2]
    maxlen_questions=tok_out[3]
    maxlen_answers=tok_out[4]
    
    return [encoder_input_data,decoder_input_data,decoder_output_data,maxlen_questions,maxlen_answers]

In [22]:
enable_gpu()

In [ ]:
Prepared_data=prepare_data(final_QA[0][0],final_QA[0][1])
print("Preparation done")
encoder_input_data=Prepared_data[0]
print("Encoder done")
decoder_input_data=Prepared_data[1]
decoder_output_data=Prepared_data[2]
print("Decoder done")
maxlen_questions=Prepared_data[3]
maxlen_answers=Prepared_data[4]
print(encoder_input_data.shape, decoder_input_data.shape, decoder_output_data.shape)
#model,encoder_inputs,encoder_states,decoder_lstm,decoder_embedding,decoder_dense,decoder_inputs
get_model_out=get_model(nb_words,embed_size,embedding_matrix)
model=get_model_out[0]
encoder_inputs=get_model_out[1]
encoder_states=get_model_out[2]
decoder_lstm=get_model_out[3]
decoder_embedding=get_model_out[4]
decoder_dense=get_model_out[5]
decoder_inputs=get_model_out[6]
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=5, epochs=10, callbacks=callbacks_list)

In [29]:
get_model_out=get_model(nb_words,embed_size,embedding_matrix)
model=get_model_out[0]
encoder_inputs=get_model_out[1]
encoder_states=get_model_out[2]
decoder_lstm=get_model_out[3]
decoder_embedding=get_model_out[4]
decoder_dense=get_model_out[5]
decoder_inputs=get_model_out[6]
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#del(decoder_output_data)
#del(decoder_input_data)
#del(encoder_input_data)
#del(chats)
#del(answers)
#del(questions)

In [ ]:
# Saving all the arrays to storage
#np.save( 'enc_in_data_PD.npy' , encoder_input_data )
#np.save( 'dec_in_data_PD.npy' , decoder_input_data )
#np.save( 'dec_tar_data_PD.npy' , decoder_output_data )

In [ ]:
#encoder_input_data=np.load("enc_in_data_PD.npy")
#decoder_input_data=np.load("dec_in_data_PD.npy")
#decoder_output_data=np.load("dec_tar_data_PD.npy")

In [ ]:
import time

In [25]:
# load the model
for i in range(110,120):
    Pre_D=prepare_data(final_QA[i][0],final_QA[i][1])
    encoder_input_data=Pre_D[0]
    decoder_input_data=Pre_D[1]
    decoder_output_data=Pre_D[2]
    maxlen_questions=Pre_D[3]
    maxlen_answers=Pre_D[4]
    model = models.load_model(filepath)
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
    callbacks_list = [checkpoint]
    model.fit([encoder_input_data , decoder_input_data], decoder_output_data, epochs=20, batch_size=5, callbacks=callbacks_list)
    #assert_allclose(model.predict(x_train),new_model.predict(x_train), 1e-5)
    print(str(i)+'/100 Sets Completed.')

Train on 100 samples
Epoch 1/20
 95/100 [===========================>..] - ETA: 0s - loss: 1.9709 - accuracy: 0.1467
Epoch 00001: loss improved from inf to 1.95073, saving model to model_DoctorChat.h5
100/100 [==============================] - 12s 120ms/sample - loss: 1.9507 - accuracy: 0.1465
Epoch 2/20
 95/100 [===========================>..] - ETA: 0s - loss: 1.7101 - accuracy: 0.2042
Epoch 00002: loss improved from 1.95073 to 1.71691, saving model to model_DoctorChat.h5
100/100 [==============================] - 10s 96ms/sample - loss: 1.7169 - accuracy: 0.2050
Epoch 3/20
 95/100 [===========================>..] - ETA: 0s - loss: 1.5701 - accuracy: 0.2464
Epoch 00003: loss improved from 1.71691 to 1.55856, saving model to model_DoctorChat.h5
100/100 [==============================] - 9s 94ms/sample - loss: 1.5586 - accuracy: 0.2463
Epoch 4/20
 95/100 [===========================>..] - ETA: 0s - loss: 1.4560 - accuracy: 0.2819
Epoch 00004: loss improved from 1.55856 to 1.43339, savi

100/100 [==============================] - 9s 92ms/sample - loss: 0.9627 - accuracy: 0.5334
Epoch 11/20
 95/100 [===========================>..] - ETA: 0s - loss: 0.8757 - accuracy: 0.5734
Epoch 00011: loss improved from 0.96265 to 0.89133, saving model to model_DoctorChat.h5
100/100 [==============================] - 10s 100ms/sample - loss: 0.8913 - accuracy: 0.5724
Epoch 12/20
 95/100 [===========================>..] - ETA: 0s - loss: 0.8242 - accuracy: 0.6044
Epoch 00012: loss improved from 0.89133 to 0.81897, saving model to model_DoctorChat.h5
100/100 [==============================] - 9s 95ms/sample - loss: 0.8190 - accuracy: 0.6003
Epoch 13/20
 95/100 [===========================>..] - ETA: 0s - loss: 0.7562 - accuracy: 0.6389
Epoch 00013: loss improved from 0.81897 to 0.74987, saving model to model_DoctorChat.h5
100/100 [==============================] - 10s 97ms/sample - loss: 0.7499 - accuracy: 0.6461
Epoch 14/20
 95/100 [===========================>..] - ETA: 0s - loss: 0.6

 95/100 [===========================>..] - ETA: 0s - loss: 0.4271 - accuracy: 0.8044
Epoch 00020: loss improved from 0.46573 to 0.41785, saving model to model_DoctorChat.h5
100/100 [==============================] - 10s 95ms/sample - loss: 0.4179 - accuracy: 0.8038
112/100 Sets Completed.
Train on 100 samples
Epoch 1/20
 95/100 [===========================>..] - ETA: 0s - loss: 2.2547 - accuracy: 0.1120
Epoch 00001: loss improved from inf to 2.25203, saving model to model_DoctorChat.h5
100/100 [==============================] - 13s 128ms/sample - loss: 2.2520 - accuracy: 0.1124
Epoch 2/20
 95/100 [===========================>..] - ETA: 0s - loss: 2.0017 - accuracy: 0.1649
Epoch 00002: loss improved from 2.25203 to 1.98392, saving model to model_DoctorChat.h5
100/100 [==============================] - 10s 97ms/sample - loss: 1.9839 - accuracy: 0.1644
Epoch 3/20
 95/100 [===========================>..] - ETA: 0s - loss: 1.7954 - accuracy: 0.2123
Epoch 00003: loss improved from 1.98392 to

100/100 [==============================] - 10s 96ms/sample - loss: 0.4765 - accuracy: 0.7601
Epoch 19/20
 95/100 [===========================>..] - ETA: 0s - loss: 0.4381 - accuracy: 0.7800
Epoch 00019: loss improved from 0.47647 to 0.42891, saving model to model_DoctorChat.h5
100/100 [==============================] - 10s 98ms/sample - loss: 0.4289 - accuracy: 0.7796
Epoch 20/20
 95/100 [===========================>..] - ETA: 0s - loss: 0.3636 - accuracy: 0.8133
Epoch 00020: loss improved from 0.42891 to 0.38385, saving model to model_DoctorChat.h5
100/100 [==============================] - 10s 103ms/sample - loss: 0.3839 - accuracy: 0.8034
115/100 Sets Completed.
Train on 100 samples
Epoch 1/20
 95/100 [===========================>..] - ETA: 0s - loss: 2.1610 - accuracy: 0.1220
Epoch 00001: loss improved from inf to 2.21226, saving model to model_DoctorChat.h5
100/100 [==============================] - 12s 124ms/sample - loss: 2.2123 - accuracy: 0.1202
Epoch 2/20
 95/100 [===========

Epoch 8/20
 95/100 [===========================>..] - ETA: 0s - loss: 1.3174 - accuracy: 0.4015
Epoch 00008: loss improved from 1.41537 to 1.31204, saving model to model_DoctorChat.h5
100/100 [==============================] - 10s 98ms/sample - loss: 1.3120 - accuracy: 0.4010
Epoch 9/20
 95/100 [===========================>..] - ETA: 0s - loss: 1.2166 - accuracy: 0.4432
Epoch 00009: loss improved from 1.31204 to 1.21913, saving model to model_DoctorChat.h5
100/100 [==============================] - 10s 98ms/sample - loss: 1.2191 - accuracy: 0.4415
Epoch 10/20
 95/100 [===========================>..] - ETA: 0s - loss: 1.1457 - accuracy: 0.4810
Epoch 00010: loss improved from 1.21913 to 1.12960, saving model to model_DoctorChat.h5
100/100 [==============================] - 10s 97ms/sample - loss: 1.1296 - accuracy: 0.4840
Epoch 11/20
 95/100 [===========================>..] - ETA: 0s - loss: 1.0519 - accuracy: 0.5138
Epoch 00011: loss improved from 1.12960 to 1.04329, saving model to mod

100/100 [==============================] - 10s 97ms/sample - loss: 0.6500 - accuracy: 0.7112
Epoch 18/20
 95/100 [===========================>..] - ETA: 0s - loss: 0.5767 - accuracy: 0.7417
Epoch 00018: loss improved from 0.65004 to 0.58860, saving model to model_DoctorChat.h5
100/100 [==============================] - 10s 96ms/sample - loss: 0.5886 - accuracy: 0.7372
Epoch 19/20
 95/100 [===========================>..] - ETA: 0s - loss: 0.5292 - accuracy: 0.7705
Epoch 00019: loss improved from 0.58860 to 0.53283, saving model to model_DoctorChat.h5
100/100 [==============================] - 10s 102ms/sample - loss: 0.5328 - accuracy: 0.7674
Epoch 20/20
 95/100 [===========================>..] - ETA: 0s - loss: 0.4827 - accuracy: 0.7947
Epoch 00020: loss improved from 0.53283 to 0.47906, saving model to model_DoctorChat.h5
100/100 [==============================] - 10s 101ms/sample - loss: 0.4791 - accuracy: 0.7940
118/100 Sets Completed.
Train on 100 samples
Epoch 1/20
 95/100 [======

In [ ]:
model.save( r'model_DocChat1.h5' ) 

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
Loaded_model = load_model( r'model_DocChat1.h5')

In [ ]:
print(Loaded_model.summary())

In [26]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [27]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen , padding='post')

In [30]:
enc_model , dec_model = make_inference_models()

# Real time testing

In [ ]:
for _ in range(10):
    try:
        states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
        empty_target_seq = np.zeros( ( 1 , 1 ) )
        empty_target_seq[0, 0] = tokenizer.word_index['start']
        stop_condition = False
        decoded_translation = ''
        while not stop_condition :
            dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
            sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
            sampled_word = None
            for word , index in tokenizer.word_index.items() :
                if sampled_word_index == index :
                    decoded_translation += ' {}'.format( word )
                    sampled_word = word
            
            if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
                stop_condition = True
                
            empty_target_seq = np.zeros( ( 1 , 1 ) )  
            empty_target_seq[ 0 , 0 ] = sampled_word_index
            states_values = [ h , c ] 

        print( " ".join(decoded_translation.strip().split(" ")[:-1]) )
    except:
        print("Sorry, I don't understand. Please try again.")